## system configuration

In [2]:
from cobra.io import read_sbml_model
model = read_sbml_model('iNF517.xml')

In [3]:
model.optimize()

,fluxes,reduced_costs
ACALDt,-0.162725,-0.0
ACMANApts,0.000000,0.0
ACTNdiff,0.000000,0.0
ACt2r,-0.830000,-0.0
ADEt2,0.010140,0.0
...,...,...
MCMAT8,0.003012,0.0
METDabc,0.000000,0.0
PSERT,0.000000,0.0
BGLA1,0.000000,0.0


In [4]:
%%time
copy_of_model = model.copy()

CPU times: user 296 ms, sys: 23.2 ms, total: 319 ms
Wall time: 320 ms


In [ ]:
%%time
for gene in model.genes:
    mutant = model.copy()
    mutant.genes.get_by_id(gene.id).knock_out()

In [ ]:
%%time
with model:
    for gene in model.genes:
        gene.knock_out()

In [ ]:
with model:
    model.reactions.PGK.bounds = 0, 0
    print("PGK's bounds inside the with statement")
    print(model.reactions.PGK.lower_bound, model.reactions.PGK.bounds)
    print('Mutant growth rate: ', model.optimize().objective_value)
print("PGK's bounds outside the with statement")
print(model.reactions.PGK.bounds)

In [ ]:
%%time
solution = model.optimize()
solution.fluxes['ATPM']

In [ ]:
%%time
model.slim_optimize()
model.reactions.ATPM.flux

In [ ]:
model.medium

In [ ]:
solution = model.optimize()

In [ ]:
solution.fluxes

## medium with glucose

In [71]:
medium = model.medium
with model:
    medium['EX_glc__D_e'] = 2.12
    model.medium = medium
    solution = model.optimize()
with model:
    # calculating max yield of etoh
    substrate_flux = model.reactions.EX_glc__D_e.flux
    model.objective = model.reactions.EX_etoh_e 
    max_etoh_e_production = model.optimize().objective_value
    print("the flux of etoh_e production: ",max_etoh_e_production , "[mmol gDW^-1 h^-1]")
    # calculating molar yield
    molar_yield_etoh_e = max_etoh_e_production / (-1. * substrate_flux)
    print("The molar yield of etoh_e ", molar_yield_etoh_e, "[mmol-etoh_e / mmol-glc]")
with model:
    # calculating max yield of etoh
    substrate_flux = model.reactions.EX_glc__D_e.flux
    model.objective = model.reactions.BIOMASS_LLA
    max_biomass_production = model.optimize().objective_value
    print("the flux of biomass production: ",max_biomass_production , "[mmol gDW^-1 h^-1]")
    # calculating molar yield
    molar_yield_biomass = max_biomass_production / (-1. * substrate_flux)
    print("The molar yield of biomass ", molar_yield_biomass, "[mmol-biomass / mmol-glc]")

the flux of etoh_e production:  1.02 [mmol gDW^-1 h^-1]
The molar yield of etoh_e  0.4811320754716981 [mmol-etoh_e / mmol-glc]
the flux of biomass production:  0.04263460544337321 [mmol gDW^-1 h^-1]
The molar yield of biomass  0.020110662944987364 [mmol-biomass / mmol-glc]


#### with 2.12 mmol / gDW*h glucose, the yiled of ethanol is 1.02 mmol / gDW*h 

## medium without glucose

In [ ]:
medium = model.medium
with model:
    medium['EX_glc__D_e'] =0
    model.medium = medium
    solution = model.optimize()
    print(solution.fluxes['EX_etoh_e'])
    
medium = model.medium
with model:
    medium['EX_glc__D_e'] =0
    model.medium = medium
    solution = model.optimize()
    print(solution.fluxes['BIOMASS_LLA'])

This result showed that L.lactics can not grow and produce ethanol without glucose

## Minimal media

In [16]:
from cobra.medium import minimal_medium

max_growth = model.slim_optimize()
minimal_medium(model, max_growth)

EX_ade_e        4.750348e-03
EX_arg__L_e     2.000000e-02
EX_asp__L_e     1.000000e-02
EX_cys__L_e     1.236698e-02
EX_glc__D_e     1.264662e+00
EX_glu__L_e     5.000000e-02
EX_gua_e        5.056891e-03
EX_his__L_e     3.223944e-03
EX_ile__L_e     1.000000e-02
EX_leu__L_e     1.576150e-02
EX_lys__L_e     1.515899e-02
EX_nac_e        8.526921e-05
EX_pi_e         2.939315e-02
EX_pnto__R_e    8.526921e-06
EX_ser__L_e     6.774111e-02
EX_thm_e        4.263461e-07
EX_thr__L_e     2.000000e-02
EX_ura_e        7.468730e-03
EX_val__L_e     1.000000e-02
dtype: float64

In [72]:
#this is for glucose content with minium medium
medium = model.medium
with model:
    medium['EX_glc__D_e'] = 1.26
    model.medium = medium
    solution = model.optimize()
medium = model.medium
with model:
    medium['EX_glc__D_e'] = 1.26
    model.medium = medium
    solution = model.optimize()

with model:
    # calculating max yield of etoh
    substrate_flux = model.reactions.EX_glc__D_e.flux
    model.objective = model.reactions.EX_etoh_e 
    max_etoh_e_production = model.optimize().objective_value
    print("the flux of etoh_e production: ",max_etoh_e_production , "[mmol gDW^-1 h^-1]")
    # calculating molar yield
    molar_yield_etoh_e = max_etoh_e_production / (-1. * substrate_flux)
    print("The molar yield of etoh_e ", molar_yield_etoh_e, "[mmol-etoh_e / mmol-glc]")   
with model:
    # calculating max yield of etoh
    substrate_flux = model.reactions.EX_glc__D_e.flux
    model.objective = model.reactions.BIOMASS_LLA
    max_biomass_production = model.optimize().objective_value
    print("the flux of biomass production: ",max_biomass_production , "[mmol gDW^-1 h^-1]")
    # calculating molar yield
    molar_yield_biomass = max_biomass_production / (-1. * substrate_flux)
    print("The molar yield of biomass ", molar_yield_biomass, "[mmol-biomass / mmol-glc]")


the flux of etoh_e production:  1.02 [mmol gDW^-1 h^-1]
The molar yield of etoh_e  0.8095238095238095 [mmol-etoh_e / mmol-glc]
the flux of biomass production:  0.042634605443373275 [mmol gDW^-1 h^-1]
The molar yield of biomass  0.020110662944987392 [mmol-biomass / mmol-glc]


## medium with glucose and other carbon source

In [64]:
#this is for glucose mix with lactose for carbon source
medium = model.medium
with model:
    medium['EX_glc__D_e'] = 1.26
    medium['EX_lcts_e'] = 0.5
    model.medium = medium
    solution = model.optimize()
medium = model.medium
with model:
    medium['EX_glc__D_e'] = 1.26
    medium['EX_lcts_e'] = 0.5
    model.medium = medium
    solution = model.optimize()
with model:
    # calculating max yield of etoh
    substrate_flux = model.reactions.EX_glc__D_e.flux + model.reactions.EX_lcts_e.flux
    model.objective = model.reactions.EX_etoh_e 
    max_etoh_e_production = model.optimize().objective_value 
    print("the flux of etoh_e production: ",max_etoh_e_production , "[mmol gDW^-1 h^-1]")
    # calculating molar yield
    molar_yield_etoh_e = max_etoh_e_production / (-1. * substrate_flux)
    print("The molar yield of etoh_e ", molar_yield_etoh_e, "[mmol-etoh_e / mmol-sub]")
with model:
    # calculating max yield of etoh
    substrate_flux = model.reactions.EX_glc__D_e.flux + model.reactions.EX_lcts_e.flux
    model.objective = model.reactions.BIOMASS_LLA
    max_biomass_production = model.optimize().objective_value
    print("the flux of biomass production: ",max_biomass_production , "[mmol gDW^-1 h^-1]")
    # calculating molar yield
    molar_yield_biomass = max_biomass_production / (-1. * substrate_flux)
    print("The molar yield of biomass ", molar_yield_biomass, "[mmol-biomass / mmol-sub]")

the flux of etoh_e production:  1.02 [mmol gDW^-1 h^-1]
The molar yield of etoh_e  0.584550362395441 [mmol-etoh_e / mmol-sub]
the flux of biomass production:  0.042634605443374614 [mmol gDW^-1 h^-1]
The molar yield of biomass  0.020110662944988023 [mmol-biomass / mmol-sub]


In [66]:
#this is for glucose mix with maltose for carbon source
medium = model.medium
with model:
    medium['EX_glc__D_e'] = 1.26
    medium['EX_malt_e'] = 0.5
    model.medium = medium
    solution = model.optimize()
medium = model.medium
with model:
    medium['EX_glc__D_e'] = 1.26
    medium['EX_malt_e'] = 0.5
    model.medium = medium
    solution = model.optimize()
with model:
    # calculating max yield of etoh
    substrate_flux = model.reactions.EX_glc__D_e.flux + model.reactions.EX_malt_e.flux
    model.objective = model.reactions.EX_etoh_e 
    max_etoh_e_production = model.optimize().objective_value
    print("the flux of etoh_e production: ",max_etoh_e_production , "[mmol gDW^-1 h^-1]")
    # calculating molar yield
    molar_yield_etoh_e = max_etoh_e_production / (-1. * substrate_flux)
    print("The molar yield of etoh_e ", molar_yield_etoh_e, "[mmol-etoh_e / mmol-glc]")
with model:
    # calculating max yield of etoh
    substrate_flux = model.reactions.EX_glc__D_e.flux + model.reactions.EX_malt_e.flux
    model.objective = model.reactions.BIOMASS_LLA
    max_biomass_production = model.optimize().objective_value
    print("the flux of biomass production: ",max_biomass_production , "[mmol gDW^-1 h^-1]")
    # calculating molar yield
    molar_yield_biomass = max_biomass_production / (-1. * substrate_flux)
    print("The molar yield of biomass ", molar_yield_biomass, "[mmol-biomass / mmol-glc]")

the flux of etoh_e production:  1.02 [mmol gDW^-1 h^-1]
The molar yield of etoh_e  0.6027042731673039 [mmol-etoh_e / mmol-glc]
the flux of biomass production:  0.04263460544337314 [mmol gDW^-1 h^-1]
The molar yield of biomass  0.02011066294498733 [mmol-biomass / mmol-glc]


In [68]:
#this is for glucose mix with frutose for carbon source
medium = model.medium
with model:
    medium['EX_glc__D_e'] = 1.26
    medium['EX_fru_e'] = 0.5
    model.medium = medium
    solution = model.optimize()
    
medium = model.medium
with model:
    medium['EX_glc__D_e'] = 1.26
    medium['EX_fru_e'] = 0.5
    model.medium = medium
    solution = model.optimize()
with model:
    # calculating max yield of etoh
    substrate_flux = model.reactions.EX_glc__D_e.flux + model.reactions.EX_fru_e.flux
    model.objective = model.reactions.EX_etoh_e 
    max_etoh_e_production = model.optimize().objective_value
    print("the flux of etoh_e production: ",max_etoh_e_production , "[mmol gDW^-1 h^-1]")
    # calculating molar yield
    molar_yield_etoh_e = max_etoh_e_production / (-1. * substrate_flux)
    print("The molar yield of etoh_e ", molar_yield_etoh_e, "[mmol-etoh_e / mmol-sub]")
with model:
    # calculating max yield of etoh
    substrate_flux = model.reactions.EX_glc__D_e.flux + model.reactions.EX_fru_e.flux
    model.objective = model.reactions.BIOMASS_LLA
    max_biomass_production = model.optimize().objective_value
    print("the flux of biomass production: ",max_biomass_production , "[mmol gDW^-1 h^-1]")
    # calculating molar yield
    molar_yield_biomass = max_biomass_production / (-1. * substrate_flux)
    print("The molar yield of biomass ", molar_yield_biomass, "[mmol-biomass / mmol-sub]")

the flux of etoh_e production:  1.02 [mmol gDW^-1 h^-1]
The molar yield of etoh_e  0.5795454545454546 [mmol-etoh_e / mmol-sub]
the flux of biomass production:  0.042634605443373046 [mmol gDW^-1 h^-1]
The molar yield of biomass  0.020110662944987284 [mmol-biomass / mmol-sub]


In [73]:
#this is for glucose mix with surcose for carbon source
medium = model.medium
with model:
    medium['EX_glc__D_e'] = 1.26
    medium['EX_sucr_e'] = 0.5
    model.medium = medium
    solution = model.optimize()

medium = model.medium
with model:
    medium['EX_glc__D_e'] = 1.26
    medium['EX_sucr_e'] = 0.5
    model.medium = medium
    solution = model.optimize()
with model:
    # calculating max yield of etoh
    substrate_flux = model.reactions.EX_glc__D_e.flux + model.reactions.EX_sucr_e.flux
    model.objective = model.reactions.EX_etoh_e 
    max_etoh_e_production = model.optimize().objective_value
    print("the flux of etoh_e production: ",max_etoh_e_production , "[mmol gDW^-1 h^-1]")
    # calculating molar yield
    molar_yield_etoh_e = max_etoh_e_production / (-1. * substrate_flux)
    print("The molar yield of etoh_e ", molar_yield_etoh_e, "[mmol-etoh_e / mmol-sub]")
with model:
    # calculating max yield of etoh
    substrate_flux = model.reactions.EX_glc__D_e.flux + model.reactions.EX_sucr_e.flux
    model.objective = model.reactions.BIOMASS_LLA
    max_biomass_production = model.optimize().objective_value
    print("the flux of biomass production: ",max_biomass_production , "[mmol gDW^-1 h^-1]")
    # calculating molar yield
    molar_yield_biomass = max_biomass_production / (-1. * substrate_flux)
    print("The molar yield of biomass ", molar_yield_biomass, "[mmol-biomass / mmol-sub]")

the flux of etoh_e production:  1.02 [mmol gDW^-1 h^-1]
The molar yield of etoh_e  0.5795454545454546 [mmol-etoh_e / mmol-sub]
the flux of biomass production:  0.042634605443373164 [mmol gDW^-1 h^-1]
The molar yield of biomass  0.02011066294498734 [mmol-biomass / mmol-sub]


In [75]:
#this is for glucose mix with galacose for carbon source
medium = model.medium
with model:
    medium['EX_glc__D_e'] = 1.26
    medium['EX_gal_e'] = 0.5
    model.medium = medium
    solution = model.optimize()

medium = model.medium
with model:
    medium['EX_glc__D_e'] = 1.26
    medium['EX_gal_e'] = 0.5
    model.medium = medium
    solution = model.optimize()

with model:
    # calculating max yield of etoh
    substrate_flux = model.reactions.EX_glc__D_e.flux + model.reactions.EX_gal_e.flux
    model.objective = model.reactions.EX_etoh_e 
    max_etoh_e_production = model.optimize().objective_value
    print("the flux of etoh_e production: ",max_etoh_e_production , "[mmol gDW^-1 h^-1]")
    # calculating molar yield
    molar_yield_etoh_e = max_etoh_e_production / (-1. * substrate_flux)
    print("The molar yield of etoh_e ", molar_yield_etoh_e, "[mmol-etoh_e / mmol-sub]")
with model:
    # calculating max yield of etoh
    substrate_flux = model.reactions.EX_glc__D_e.flux + model.reactions.EX_gal_e.flux
    model.objective = model.reactions.BIOMASS_LLA
    max_biomass_production = model.optimize().objective_value
    print("the flux of biomass production: ",max_biomass_production , "[mmol gDW^-1 h^-1]")
    # calculating molar yield
    molar_yield_biomass = max_biomass_production / (-1. * substrate_flux)
    print("The molar yield of biomass ", molar_yield_biomass, "[mmol-biomass / mmol-sub]")

the flux of etoh_e production:  1.02 [mmol gDW^-1 h^-1]
The molar yield of etoh_e  0.5795454545454546 [mmol-etoh_e / mmol-sub]
the flux of biomass production:  0.04263460544337336 [mmol gDW^-1 h^-1]
The molar yield of biomass  0.020110662944987433 [mmol-biomass / mmol-sub]


In [77]:
#this is for glucose mix with glycerol for carbon source
medium = model.medium
with model:
    medium['EX_glc__D_e'] = 1.26
    medium['EX_glyc_e'] = 0.5
    model.medium = medium
    solution = model.optimize()
    
    
medium = model.medium
with model:
    medium['EX_glc__D_e'] = 1.26
    medium['EX_glyc_e'] = 0.5
    model.medium = medium
    solution = model.optimize()

with model:
    # calculating max yield of etoh
    substrate_flux = model.reactions.EX_glc__D_e.flux + model.reactions.EX_glyc_e.flux
    model.objective = model.reactions.EX_etoh_e 
    max_etoh_e_production = model.optimize().objective_value
    print("the flux of etoh_e production: ",max_etoh_e_production , "[mmol gDW^-1 h^-1]")
    # calculating molar yield
    molar_yield_etoh_e = max_etoh_e_production / (-1. * substrate_flux)
    print("The molar yield of etoh_e ", molar_yield_etoh_e, "[mmol-etoh_e / mmol-sub]")
with model:
    # calculating max yield of etoh
    substrate_flux = model.reactions.EX_glc__D_e.flux + model.reactions.EX_glyc_e.flux
    model.objective = model.reactions.BIOMASS_LLA
    max_biomass_production = model.optimize().objective_value
    print("the flux of biomass production: ",max_biomass_production , "[mmol gDW^-1 h^-1]")
    # calculating molar yield
    molar_yield_biomass = max_biomass_production / (-1. * substrate_flux)
    print("The molar yield of biomass ", molar_yield_biomass, "[mmol-biomass / mmol-sub]")

the flux of etoh_e production:  1.02 [mmol gDW^-1 h^-1]
The molar yield of etoh_e  0.6108062317430152 [mmol-etoh_e / mmol-sub]
the flux of biomass production:  0.04263460544337352 [mmol gDW^-1 h^-1]
The molar yield of biomass  0.02582753440462172 [mmol-biomass / mmol-sub]


## in anaerobic condition

In [79]:
#this is for anaerobic condition 
medium = model.medium
with model:
    medium['EX_glc__D_e'] = 1.26
    medium['EX_o2_e'] = 0
    model.medium = medium
    solution = model.optimize()

medium = model.medium
with model:
    medium['EX_glc__D_e'] = 1.26
    medium['EX_o2_e'] = 0
    model.medium = medium
    solution = model.optimize()
    
with model:
    # calculating max yield of etoh
    substrate_flux = model.reactions.EX_glc__D_e.flux + model.reactions.EX_o2_e.flux
    model.objective = model.reactions.EX_etoh_e 
    max_etoh_e_production = model.optimize().objective_value
    print("the flux of etoh_e production: ",max_etoh_e_production , "[mmol gDW^-1 h^-1]")
    # calculating molar yield
    molar_yield_etoh_e = max_etoh_e_production / (-1. * substrate_flux)
    print("The molar yield of etoh_e ", molar_yield_etoh_e, "[mmol-etoh_e / mmol-sub]")
with model:
    # calculating max yield of etoh
    substrate_flux = model.reactions.EX_glc__D_e.flux + model.reactions.EX_o2_e.flux
    model.objective = model.reactions.BIOMASS_LLA
    max_biomass_production = model.optimize().objective_value
    print("the flux of biomass production: ",max_biomass_production , "[mmol gDW^-1 h^-1]")
    # calculating molar yield
    molar_yield_biomass = max_biomass_production / (-1. * substrate_flux)
    print("The molar yield of biomass ", molar_yield_biomass, "[mmol-biomass / mmol-sub]")

the flux of etoh_e production:  1.02 [mmol gDW^-1 h^-1]
The molar yield of etoh_e  0.8095238095238095 [mmol-etoh_e / mmol-sub]
the flux of biomass production:  0.042634605443360146 [mmol gDW^-1 h^-1]
The molar yield of biomass  0.0201106629449812 [mmol-biomass / mmol-sub]


## in aerobic condition

In [84]:
#this is for aerobic condition
medium = model.medium
with model:
    medium['EX_glc__D_e'] = 1.26
    medium['EX_o2_e'] = 5
    model.medium = medium
    solution = model.optimize()
   
medium = model.medium
with model:
    medium['EX_glc__D_e'] = 1.26
    medium['EX_o2_e'] = 5
    model.medium = medium
    solution = model.optimize()
with model:
    # calculating max yield of etoh
    substrate_flux = model.reactions.EX_glc__D_e.flux + model.reactions.EX_o2_e.flux
    model.objective = model.reactions.EX_etoh_e 
    max_etoh_e_production = model.optimize().objective_value
    print("the flux of etoh_e production: ",max_etoh_e_production , "[mmol gDW^-1 h^-1]")
    # calculating molar yield
    molar_yield_etoh_e = max_etoh_e_production / (-1. * substrate_flux)
    print("The molar yield of etoh_e ", molar_yield_etoh_e, "[mmol-etoh_e / mmol-sub]")
with model:
    # calculating max yield of etoh
    substrate_flux = model.reactions.EX_glc__D_e.flux + model.reactions.EX_o2_e.flux
    model.objective = model.reactions.BIOMASS_LLA
    max_biomass_production = model.optimize().objective_value
    print("the flux of biomass production: ",max_biomass_production , "[mmol gDW^-1 h^-1]")
    # calculating molar yield
    molar_yield_biomass = max_biomass_production / (-1. * substrate_flux)
    print("The molar yield of biomass ", molar_yield_biomass, "[mmol-biomass / mmol-sub]")

the flux of etoh_e production:  1.02 [mmol gDW^-1 h^-1]
The molar yield of etoh_e  0.8095238095238095 [mmol-etoh_e / mmol-sub]
the flux of biomass production:  0.04263460544337292 [mmol gDW^-1 h^-1]
The molar yield of biomass  0.020110662944987225 [mmol-biomass / mmol-sub]
